In [13]:
import numpy as np
import cv2

In [14]:
# Read the video input
vid = cv2.VideoCapture('./Data/test.avi')

# Get the frame count
n_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

# Width and height of each frame
w = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# FPS
fps = vid.get(cv2.CAP_PROP_FPS)

# Set up output
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# Output format
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('test_out.mp4', fourcc, fps, (w, h))

In [15]:
# Read first frame
_, prev = vid.read()

# Convert to Grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

In [16]:
# Initialize the transformation matrix
transforms = np.zeros((n_frames - 1, 3), np.float32)

for i in range(n_frames - 1):
    # Feature points of the previous frame
    # The feature points are obtained using the Shi-Tomasi corner detection algorithm
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)

    # Read the next frame, if it doesn't exist, exit the loop
    flag, frame = vid.read()
    if not flag:
        break

    # Convert to Grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate the Optical flow using the Lucas-Kanade method
    frame_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_pts, None)

    # Sanity check
    assert frame_pts.shape == prev_pts.shape

    # Only store the valid pts
    idx = np.where(status == 1)[0]
    prev_pts = prev_pts[idx]
    frame_pts = frame_pts[idx]

    # Obtain the transformation matrix
    mat = cv2.estimateAffine2D(prev_pts, frame_pts)[0]

    # Translation components
    dx = mat[0][2]
    dy = mat[1][2]

    # Rotation component
    dtheta = np.arctan2(mat[1, 0], mat[0, 0])

    # Store the components corresponding to each frame
    transforms[i] = [dx ,dy, dtheta]

    # Store the current frame
    prev_gray = frame_gray

In [17]:
# Obtain the trajectory as a cumulative sum of the dx, dy and dtheta components
trajectory = np.cumsum(transforms, axis=0)

In [18]:
# Moving average filter
def mav_filter(curve, radius):
    window_size = 2*radius + 1
    filter = np.ones(window_size) / window_size
    # Padding
    curve_padded = np.lib.pad(curve, (radius, radius), 'edge')
    curve_smooth = np.convolve(curve_padded, filter, mode='same')
    # Remove padding
    curve_output = curve_smooth[radius:-radius]

    return curve_output

In [19]:
# Smoothen the trajectory
def smooth(trajectory, radius=7):
    trajectory_smooth = np.copy(trajectory)

    for i in range(3):
        trajectory_smooth[:, i] = mav_filter(trajectory[:, i], radius)

    return trajectory_smooth

In [20]:
from scipy.special import comb as n_over_k

def BezierFit(trajectory):
    Mtk = lambda n, t, k: t**k * (1-t)**(n-k) * n_over_k(n,k)
    BezierCoeff = lambda ts: [[Mtk(3,t,k) for k in range(4)] for t in ts]

    fcn = np.log
    tPlot = np.linspace(0. ,1. , 81)
    xPlot = np.linspace(0.1,2.5, 81)
    tData = tPlot[0:81:10]
    xData = xPlot[0:81:10]
    data = np.column_stack((xData, fcn(xData))) # shapes (9,2)

    Pseudoinverse = np.linalg.pinv(BezierCoeff(tData)) # (9,4) -> (4,9)
    control_points = Pseudoinverse.dot(data)     # (4,9)*(9,2) -> (4,2)
    Bezier = np.array(BezierCoeff(tPlot)).dot(control_points)

In [21]:
"""
Code for a Catmull-Rom spline curve fit
Obtained from: https://github.com/vmichals/python-algos/blob/master/catmull_rom_spline.py
"""


def catmull_rom_one_point(x, v0, v1, v2, v3):
    """Computes interpolated y-coord for given x-coord using Catmull-Rom.
    Computes an interpolated y-coordinate for the given x-coordinate between
    the support points v1 and v2. The neighboring support points v0 and v3 are
    used by Catmull-Rom to ensure a smooth transition between the spline
    segments.
    Args:
        x: the x-coord, for which the y-coord is needed
        v0: 1st support point
        v1: 2nd support point
        v2: 3rd support point
        v3: 4th support point
    """
    c1 = 1. * v1
    c2 = -.5 * v0 + .5 * v2
    c3 = 1. * v0 + -2.5 * v1 + 2. * v2 -.5 * v3
    c4 = -.5 * v0 + 1.5 * v1 + -1.5 * v2 + .5 * v3
    return (((c4 * x + c3) * x + c2) * x + c1)

def catmull_rom(p_x, p_y, res=100):
    """Computes Catmull-Rom Spline for given support points and resolution.
    Args:
        p_x: array of x-coords
        p_y: array of y-coords
        res: resolution of a segment (including the start point, but not the
            endpoint of the segment)
    """
    # create arrays for spline points
    x_intpol = np.empty(res*(len(p_x)-1) + 1)
    y_intpol = np.empty(res*(len(p_x)-1) + 1)

    # set the last x- and y-coord, the others will be set in the loop
    x_intpol[-1] = p_x[-1]
    y_intpol[-1] = p_y[-1]

    # loop over segments (we have n-1 segments for n points)
    for i in range(len(p_x)-1):
        # set x-coords
        x_intpol[i*res:(i+1)*res] = np.linspace(
            p_x[i], p_x[i+1], res, endpoint=False)
        if i == 0:
            # need to estimate an additional support point before the first
            y_intpol[:res] = np.array([
                catmull_rom_one_point(
                    x,
                    p_y[0] - (p_y[1] - p_y[0]), # estimated start point,
                    p_y[0],
                    p_y[1],
                    p_y[2])
                for x in np.linspace(0.,1.,res, endpoint=False)])
        elif i == len(p_x) - 2:
            # need to estimate an additional support point after the last
            y_intpol[i*res:-1] = np.array([
                catmull_rom_one_point(
                    x,
                    p_y[i-1],
                    p_y[i],
                    p_y[i+1],
                    p_y[i+1] + (p_y[i+1] - p_y[i]) # estimated end point
                ) for x in np.linspace(0.,1.,res, endpoint=False)])
        else:
            y_intpol[i*res:(i+1)*res] = np.array([
                catmull_rom_one_point(
                    x,
                    p_y[i-1],
                    p_y[i],
                    p_y[i+1],
                    p_y[i+2]) for x in np.linspace(0.,1.,res, endpoint=False)])


    return (x_intpol, y_intpol)

In [22]:
# Obtain the smooth trajectory
trajectory_smooth = smooth(trajectory)

# Fit a Catmull-Rom spline to the trajectory
frames = np.arange(n_frames - 1)
catmull_rom_spline = np.copy(trajectory_smooth)

for i in range(3):
    predicted_trajectory = catmull_rom(frames, trajectory_smooth[:, i])[1]
    for j in range(n_frames - 1):
        catmull_rom_spline[j, i] = predicted_trajectory[50*j]


# Compute the difference in trajectories
trajectory_difference = catmull_rom_spline - trajectory

# Obtain the new transformation components
transforms_smooth = transforms + trajectory_difference

In [23]:
# Fix the border by scaling the image about its center
def fix_border(frame):
    n = frame.shape
    T = cv2.getRotationMatrix2D((n[1]/2, n[0]/2), 0, 1.1)
    frame_out = cv2.warpAffine(frame, T, (n[1], n[0]))

    return frame_out

In [24]:
# Reset stream to first frame
vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Transform the frame using the new transformation components to obtain the stabilized frame
for i in range(n_frames - 1):
    flag, frame = vid.read()
    if not flag:
        break

    # Transformation components for the current matrix
    dx = transforms_smooth[i, 0]
    dy = transforms_smooth[i, 1]
    dtheta = transforms_smooth[i, 2]
    
    # Construct the transformation matrix
    mat = np.zeros((2, 3), np.float32)
    mat[0, 0] = np.cos(dtheta)
    mat[0, 1] = -np.sin(dtheta)
    mat[1, 0] = np.sin(dtheta)
    mat[1, 1] = np.cos(dtheta)
    mat[0, 2] = dx
    mat[1, 2] = dy

    # Apply the transformation
    frame_stabilized = cv2.warpAffine(frame, mat, (w, h))
    frame_stabilized = fix_border(frame_stabilized)

    # Write to file
    frame_out = cv2.hconcat([frame, frame_stabilized])
    # frame_out = frame_stabilized

    # If too big, resize
    if(frame_out.shape[1] > 1920):
        frame_out = cv2.resize(frame_out, (w, h))
    
    cv2.imshow('frame', frame_out)
    cv2.waitKey(10)
    out.write(frame_out)

vid.release()
out.release()
cv2.destroyAllWindows()